In [ ]:
!pip install kaggle



In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split



In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pips123","key":"953bc30af80e83d9ce2a3bdd8a6264da"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [151]:
!kaggle competitions download -c child-mind-institute-problematic-internet-use
!unzip child-mind-institute-problematic-internet-use.zip
!ls

child-mind-institute-problematic-internet-use.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  child-mind-institute-problematic-internet-use.zip
replace data_dictionary.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace series_test.parquet/id=00115b9f/part-0.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace series_test.parquet/id=001f3379/part-0.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace series_train.parquet/id=00115b9f/part-0.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: child-mind-institute-problematic-internet-use.zip  sample_submission.csv  test.csv
data_dictionary.csv				   series_test.parquet	  train.csv
sample_data					   series_train.parquet


In [225]:
train_csv_path = 'train.csv'
test_csv_path = 'test.csv'

train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

print("Train Data:")
print(train_data.head())

print("\nTest Data:")
print(test_data.head())

print("\nInformações do Train Data:")
print(train_data.info())

print("\nEstatísticas descritivas do Train Data:")
print(train_data.describe())

print("\nValores ausentes no Train Data:")
print(train_data.isnull().sum())

if 'sii' in train_data.columns:
    print("\nDistribuição do Alvo (sii):")
    print(train_data['sii'].value_counts())

Train Data:
         id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  ...  PCIAT-PCIAT_18  PCIAT-PCI

In [226]:
missing_data = train_data.isnull().mean().sort_values(ascending=False)
print("Proporção de valores ausentes:")
print(missing_data.head(20))


Proporção de valores ausentes:
PAQ_A-PAQ_A_Total               0.880051
PAQ_A-Season                    0.880051
Fitness_Endurance-Time_Sec      0.813131
Fitness_Endurance-Time_Mins     0.813131
Fitness_Endurance-Max_Stage     0.812374
Physical-Waist_Circumference    0.773232
FGC-FGC_GSND_Zone               0.731818
FGC-FGC_GSD_Zone                0.731566
FGC-FGC_GSD                     0.728788
FGC-FGC_GSND                    0.728788
Fitness_Endurance-Season        0.669697
PAQ_C-Season                    0.565404
PAQ_C-PAQ_C_Total               0.565404
BIA-BIA_DEE                     0.497222
BIA-BIA_FFM                     0.497222
BIA-BIA_BMR                     0.497222
BIA-BIA_BMI                     0.497222
BIA-BIA_ECW                     0.497222
BIA-BIA_FFMI                    0.497222
BIA-BIA_FMI                     0.497222
dtype: float64


In [227]:
threshold = 0.7
columns_to_drop = missing_data[missing_data > threshold].index

train_data_cleaned = train_data.drop(columns=columns_to_drop)
print(f"Colunas removidas: {list(columns_to_drop)}")


Colunas removidas: ['PAQ_A-PAQ_A_Total', 'PAQ_A-Season', 'Fitness_Endurance-Time_Sec', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Max_Stage', 'Physical-Waist_Circumference', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSND']


In [228]:
print("Valores ausentes restantes:")
print(train_data_cleaned.isnull().sum().sum())

missing_data_remaining = train_data_cleaned.isnull().mean().sort_values(ascending=False)
print("Proporção de valores ausentes restantes:")
print(missing_data_remaining.head(20))


Valores ausentes restantes:
100461
Proporção de valores ausentes restantes:
Fitness_Endurance-Season      0.669697
PAQ_C-PAQ_C_Total             0.565404
PAQ_C-Season                  0.565404
BIA-BIA_Frame_num             0.497222
BIA-BIA_FMI                   0.497222
BIA-BIA_BMR                   0.497222
BIA-BIA_DEE                   0.497222
BIA-BIA_ECW                   0.497222
BIA-BIA_FFM                   0.497222
BIA-BIA_FFMI                  0.497222
BIA-BIA_Fat                   0.497222
BIA-BIA_Activity_Level_num    0.497222
BIA-BIA_ICW                   0.497222
BIA-BIA_LDM                   0.497222
BIA-BIA_LST                   0.497222
BIA-BIA_SMM                   0.497222
BIA-BIA_TBW                   0.497222
BIA-BIA_BMC                   0.497222
BIA-BIA_BMI                   0.497222
BIA-Season                    0.458333
dtype: float64


In [229]:
columns_to_remove = [
    'Fitness_Endurance-Season', 'PAQ_C-Season', 'BIA-BIA_ECW',
    'PCIAT-PCIAT_Total', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_18',
    'PCIAT-PCIAT_17', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_08',
    'PCIAT-PCIAT_20', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',
    'PCIAT-PCIAT_16',
    'PCIAT-PCIAT_13', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_14',
    'PCIAT-PCIAT_19', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_06',
    'PCIAT-PCIAT_01', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_04',
    'PCIAT-PCIAT_07', 'PCIAT-PCIAT_12'
]

train_data_cleaned = train_data_cleaned.drop(columns=columns_to_remove, errors='ignore')

columns_with_nulls = train_data_cleaned.columns[train_data_cleaned.isnull().any()]
numeric_columns_with_nulls = train_data_cleaned[columns_with_nulls].select_dtypes(include=['float64', 'int64']).columns
categorical_columns_with_nulls = train_data_cleaned[columns_with_nulls].select_dtypes(include=['object']).columns

imputer_numeric = SimpleImputer(strategy='mean')
train_data_cleaned[numeric_columns_with_nulls] = imputer_numeric.fit_transform(train_data_cleaned[numeric_columns_with_nulls])

imputer_categorical = SimpleImputer(strategy='most_frequent')
train_data_cleaned[categorical_columns_with_nulls] = imputer_categorical.fit_transform(train_data_cleaned[categorical_columns_with_nulls])

print("Valores ausentes restantes após imputação completa:")
print(train_data_cleaned.isnull().sum().sum())


Valores ausentes restantes após imputação completa:
0


In [230]:
categorical_columns = train_data_cleaned.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    train_data_cleaned[col] = le.fit_transform(train_data_cleaned[col])

numeric_columns = train_data_cleaned.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
train_data_cleaned[numeric_columns] = scaler.fit_transform(train_data_cleaned[numeric_columns])

print("Dados normalizados e categóricos codificados.")


Dados normalizados e categóricos codificados.


In [231]:
X = train_data_cleaned.drop(columns=['sii'])
y = train_data_cleaned['sii']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Conjunto de treino: {X_train.shape}, Conjunto de validação: {X_val.shape}")


Conjunto de treino: (3168, 47), Conjunto de validação: (792, 47)


In [232]:
y_train_class = np.round(y_train).astype(int)
y_val_class = np.round(y_val).astype(int)


In [233]:
print("Valores únicos de y_train_class:", np.unique(y_train_class))


Valores únicos de y_train_class: [-1  0  1  2  4]


In [234]:
print("Valores menores que 0:", y_train[y_train < 0])
print("Valores maiores que 3:", y_train[y_train > 3])


Valores menores que 0: 1616   -0.905692
2408   -0.905692
2802   -0.905692
3951   -0.905692
2123   -0.905692
          ...   
3092   -0.905692
1095   -0.905692
1130   -0.905692
1294   -0.905692
3174   -0.905692
Name: sii, Length: 1283, dtype: float64
Valores maiores que 3: 2860    3.775616
1907    3.775616
1454    3.775616
370     3.775616
191     3.775616
2005    3.775616
2691    3.775616
2689    3.775616
3494    3.775616
2316    3.775616
306     3.775616
3223    3.775616
3074    3.775616
3030    3.775616
405     3.775616
1107    3.775616
2263    3.775616
255     3.775616
2036    3.775616
868     3.775616
894     3.775616
2504    3.775616
487     3.775616
2331    3.775616
2731    3.775616
1076    3.775616
Name: sii, dtype: float64


In [235]:
y_train_adjusted = y_train.clip(0, 3)
y_val_adjusted = y_val.clip(0, 3)

y_train_class = np.round(y_train_adjusted).astype(int)
y_val_class = np.round(y_val_adjusted).astype(int)

print("Valores únicos após ajuste:", np.unique(y_train_class))


Valores únicos após ajuste: [0 1 2 3]


In [236]:
print("Distribuição de classes em y_train_class:")
print(pd.Series(y_train_class).value_counts())


Distribuição de classes em y_train_class:
sii
0    2258
1     582
2     302
3      26
Name: count, dtype: int64


In [242]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, cohen_kappa_score

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train_class)

y_pred = model.predict(X_val)

print("Relatório de Classificação:")
print(classification_report(y_val_class, y_pred))

kappa = cohen_kappa_score(y_val_class, y_pred, weights='quadratic')
print(f"Quadratic Weighted Kappa: {kappa}")


Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.76      0.96      0.85       560
           1       0.32      0.14      0.20       148
           2       0.33      0.07      0.11        76
           3       0.00      0.00      0.00         8

    accuracy                           0.71       792
   macro avg       0.35      0.29      0.29       792
weighted avg       0.63      0.71      0.65       792

Quadratic Weighted Kappa: 0.26654996817313814


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [238]:
correlation = X_train.corrwith(y_train).sort_values(ascending=False)
print("Correlação com o Alvo (sii):")
print(correlation.head(10))


Correlação com o Alvo (sii):
Physical-Height                           0.321510
PreInt_EduHx-computerinternet_hoursday    0.285892
Basic_Demos-Age                           0.281538
Physical-Weight                           0.277437
SDS-SDS_Total_T                           0.218836
SDS-SDS_Total_Raw                         0.214633
Physical-BMI                              0.185064
FGC-FGC_CU                                0.171753
BIA-BIA_BMI                               0.169407
Physical-Systolic_BP                      0.134133
dtype: float64
